In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import time
import random


In [3]:
plm=pd.read_excel("C:/Users/claudio.pacheco/Downloads/plm_limpio.xlsx")

In [4]:
#Eliminar duplicados de nombre_producto
plm=plm.drop_duplicates(subset=['nombre_producto'], keep='first')
plm

,nombre_producto,sustancia,forma_farm,lab,presentacion
0,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 20 tabletas , 1.0 miligramos"
10,a.s. cor,norfenefrina,solucion,sanfer,"1 frasco con gotero, 24 ml,"
11,abacavir,abacavir,tabletas,cryopharma,"1 caja , 60 tabletas , 300 miligramos"
12,abatoarin,tropisetron,capsulas,pisa,"1 caja , 1 capsulas , 5 miligramos"
15,abbatin,montelukast,tabletas masticables,corne,"1 caja,20 tabletas masticables,5 mg"
...,...,...,...,...,...
16634,zyrtec,cetirizina,solucion,armstrong,"1 caja, 1 frasco con gotero, 10 ml, 10 mg/ml"
16648,zytiga,abiraterona,tabletas,janssen-cilag,"1 caja, 1 frasco(s), 120 tabletas, 250 miligramos"
16650,zytokil,doxorubicina,solucion inyectable,pisa,NaN
16651,zyvoxam,linezolid,solucion inyectable,pfizer,"1 sobre con bolsa de infusion, 300 ml, 2 mg/ml"


In [10]:
#Crear lista con nombre producto por grupos de 500
lista=plm['nombre_producto'].tolist()
#Hacer lista de listas separadas por 200
lista2=[lista[i:i + 200] for i in range(0, len(lista), 200)]
#Hacer lista de listas separadas por 10
lista3=[lista[i:i + 10] for i in range(0, len(lista), 10)]
#Dejar los primeros dos grupos de 10
lista3=lista3[0:2]

In [11]:
lista3

[['a.f. valdecasas',
  'a.s. cor',
  'abacavir',
  'abatoarin',
  'abbatin',
  'abelcet',
  'ab-life',
  'abretia',
  'abrixone',
  'acanol'],
 ['acarbosa',
  'acavexal',
  'acc',
  'accocept',
  'accocit',
  'accogard',
  'accogem',
  'accolate',
  'accolok',
  'accua aseptic']]

In [13]:

"""Esta función busca en la página de la COFEPRIS el registro sanitario de un medicamento
    
    """

def registro(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("/usr/local/bin/chromedriver"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame()

    soup = BeautifulSoup(table_contents, 'html.parser')
    registro=[]
    marca = []
    generica = []
    tipo_med=[]
    indica = []
    lab = []
    fabricante = []
    principio=[]

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblNumeroRegistro'):
                registro.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span["id"].startswith('MainContent_GV_RegistroSanitario_lblTipoMedicamento'):
                tipo_med.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblPrincipioActivo'):
                principio.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblIndicacionTerapeutica'):
                indica.append(span.text)

    df = pd.DataFrame({'reg_san':registro,'marca': marca, 'den_gen': generica, 'tipo_med':tipo_med, 'titular': lab, 'fabricante': fabricante, 'principio': principio,'indicacion':indica,"busqueda":medicamento})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df

In [16]:
tabla=pd.DataFrame()
for i in lista3:
    for j in i:
        #Búsqueda en una variable
        tabla=pd.concat([tabla,registro(j)])
        #Tiempo entre elementos
        time.sleep(random.randint(1,2))
        #Tiempo entre listas
    time.sleep(random.randint(1,2))
    

tabla


,reg_san,marca,den_gen,tipo_med,titular,fabricante,principio,indicacion,busqueda
0,82231 ssa,a.f. valdecasas,ácido fólico,de referencia,laboratorios valdecasas s.a.,laboratorios valdecasas s.a.,b03bb01 ácido fólico,antianémico y en mujeres de edad reproductiva ...,a.f. valdecasas
1,002v2000 ssa,a.f. valdecasas.,ácido fólico,de referencia,"laboratorios valdecasas, s. a.",laboratorios valdecasas s.a.,b03bb01 ácido fólico,"para las deficiencias del ácido fólico, preven...",a.f. valdecasas
2,045v2000 ssa,a.f. valdecasas,ácido fólico,de referencia,"laboratorios valdecasas, s. a.",laboratorios valdecasas s.a.,b03bb01 ácido fólico,para las deficiencias del ácido fólico.,a.f. valdecasas
0,136m2014 ssa,a.s. cor,norfenefrina,de referencia,"grimann, s. a. de c.v.","grimann, s.a. de c.v.",c01ca03 norepinefrina.,vasoconstrictor,a.s. cor
0,282m2018 ssa,narkson,abacavir / lamivudina,genérico,"nucitec, s.a. de c.v.","nucitec, s.a. de c.v.",,para el tratamiento de la infección causada po...,abacavir
1,048m2020 ssa,imibide,abacavir / lamivudina,genérico,,hetero labs limited,j05ar02 lamivudina y abacavir,antirretroviral.,abacavir
2,177m2021 ssa,exfhalir,abacavir / lamivudina,genérico,"farmaceutica hispanoamericana, s.a. de c.v.",productos medix s.a de c.v.,j05ar02 lamivudina y abacavir,antirretroviral. para el tratamiento por la in...,abacavir
3,076m2020 ssa,neolintra,abacavir,genérico,"neolpharma, s.a. de c.v.","neolpharma, s. a. de c. v.",j05af99 otros principios activos. agentes de ...,antirretroviral,abacavir
4,130m2021 ssa,vexakin,abacavir / lamivudina,genérico,,mylan laboratories ltd (unit-2),j05ar02 lamivudina y abacavir,antirretroviral. para el tratamiento de la inf...,abacavir
0,265m2008 ssa,abatoarin,tropisetrón,genérico,"laboratorios pisa, s.a. de c.v.","laboratorios pisa, s.a. de c.v.",a04aa03 tropisetrón.,antiemético.,abatoarin


In [ ]:
#Salvar tabla en csv
tabla.to_csv("registro_sanitario.csv",index=False)